# 🤖 Formation RAG PDF Chat - VERSION CORRIGÉE

## Guide pour l'instructeur
Ce document contient toutes les solutions des exercices. À utiliser pour aider les étudiants pendant la formation.

## Architecture du système
```
PDF → Chunking → Embeddings → Vector Store → Retrieval → LLM → Response
```

## 0. Configuration Google Colab

### Installation d'Ollama et des modèles

In [ ]:
# Installation d'Ollama sur Google Colab
import subprocess
import time
import os

print("🚀 Installation d'Ollama...")

# Installation d'Ollama
subprocess.run(["curl", "-fsSL", "https://ollama.ai/install.sh"], stdout=subprocess.PIPE)
subprocess.run(["sh", "-c", "curl -fsSL https://ollama.ai/install.sh | sh"], shell=True)

# Démarrage du service Ollama en arrière-plan
print("🔄 Démarrage du service Ollama...")
process = subprocess.Popen(["ollama", "serve"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
time.sleep(5)  # Attendre que le service démarre

print("📥 Téléchargement des modèles...")
# Pull des modèles nécessaires
subprocess.run(["ollama", "pull", "llama3.2:3b"])
subprocess.run(["ollama", "pull", "nomic-embed-text:latest"])

print("✅ Installation terminée!")

## 1. Installation des dépendances Python

In [ ]:
# Installation des dépendances
!pip install streamlit faiss-cpu numpy PyPDF2 langchain scikit-learn ollama-python

from __future__ import annotations

import os
import tempfile
from typing import Any, Dict, List, Optional, Tuple

import faiss  # type: ignore
import numpy as np
import streamlit as st
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sklearn.metrics.pairwise import cosine_similarity

import ollama

## 2. Configuration de l'Interface - SOLUTION

In [ ]:
# Configuration de la page Streamlit
st.set_page_config(page_title="🤖 RAG PDF Chat", page_icon="🤖", layout="wide")

# SOLUTION: CSS complet pour l'interface
st.markdown(
    """
<style>
html, body {
    font-family: 'Helvetica Neue', sans-serif;
    background-color: #f4f7fa;
}
.sidebar .sidebar-content {
    background-color: #ffffff;
}
.chat-container {
    max-width: 1200px;
    margin: auto;
    padding: 1rem;
}
.user-msg {
    background: #007bff;
    color: white;
    border-radius: 20px 20px 0px 20px;
    padding: 1rem;
    margin: 0.5rem 0;
    align-self: flex-end;
    max-width: 100%;
    word-wrap: break-word;
}
.bot-msg {
    background: #e9ecef;
    color: #212529;
    border-radius: 20px 20px 20px 0px;
    padding: 1rem;
    margin: 0.5rem 0;
    align-self: flex-start;
    max-width: 100%;
    word-wrap: break-word;
}
.chat-area {
    display: flex;
    flex-direction: column;
}
</style>
""",
    unsafe_allow_html=True,
)

## 3. Configuration des modèles

In [ ]:
# Configuration des modèles et paramètres
MODEL_NAME = "llama3.2:3b"
EMBEDDING_MODEL = "nomic-embed-text:latest"
DOC_TOP_K = 3
CHUNK_TOP_K = 5
CANDIDATES_K = 20
NEIGHBORS = 1
LAMBDA_DIVERSITY = 0.3
SIM_THRESHOLD = 0.25
TEMPERATURE = 0.2
MAX_TOKENS = 2048

## 4. Fonctions LLM - SOLUTION

In [ ]:
# SOLUTION: Implémentation complète des fonctions LLM
def _call_llm(messages: List[Dict[str, str]], *, temperature: float = 0.1, max_tokens: int = 2048, stream: bool = False):
    """Appel au modèle de langage via Ollama"""
    return ollama.chat(
        model=MODEL_NAME,
        messages=messages,
        stream=stream,
        options={"temperature": temperature, "num_predict": max_tokens},
    )

def embed_texts(texts: List[str]) -> np.ndarray:
    """Génère les embeddings pour une liste de textes"""
    return np.array([
        ollama.embeddings(model=EMBEDDING_MODEL, prompt=t)["embedding"] 
        for t in texts
    ], dtype="float32")

## 5. Traitement PDF

In [ ]:
def clean_text(text: str) -> str:
    """Nettoie le texte extrait du PDF"""
    import re
    text = re.sub(r"\n{2,}", "\n\n", text)
    text = re.sub(r"(\w+)-\n(\w+)", r"\1\2", text)
    text = re.sub(r"\s{2,}", " ", text)
    return text.strip()

def extract_pdf_text(path: str) -> str:
    """Extrait le texte d'un fichier PDF"""
    return clean_text("\n".join(page.extract_text() or "" for page in PdfReader(path).pages))

## 6. Chunking - SOLUTION

In [ ]:
# SOLUTION: Implémentation du chunking adaptatif
def auto_chunk_size(tokens: int) -> int:
    """Détermine la taille optimale des chunks"""
    return 1024 if tokens < 8000 else 768 if tokens < 20000 else 512

def chunk_document(text: str) -> List[str]:
    """Segmente un document en chunks avec chevauchement"""
    size = auto_chunk_size(len(text.split()))
    
    splitter = RecursiveCharacterTextSplitter(
        separators=["\n\n", "\n", ". "],
        chunk_size=size,
        chunk_overlap=int(size*0.25),
        length_function=len,
    )
    
    return [c for c in splitter.split_text(text) if len(c) > 100]

## 7. Génération de résumés

In [ ]:
def make_summary(text: str) -> str:
    """Génère un résumé structuré du document"""
    messages = [
        {
            "role": "system", 
            "content": "Vous êtes un expert en synthèse documentaire. Résumez le texte suivant en trois parties : (1) Contexte, (2) Points clés, (3) Conclusions. Répondez en français."
        },
        {"role": "user", "content": text[:120000]}
    ]
    return _call_llm(messages)["message"]["content"].strip()

## 8. Classe RagIndex - SOLUTION COMPLÈTE

In [ ]:
# SOLUTION: Classe RagIndex complète
class RagIndex:
    """Classe principale pour l'indexation et la recherche RAG"""
    
    def __init__(self):
        self.doc_index: Optional[faiss.IndexFlatIP] = None
        self.chunk_index: Optional[faiss.Index] = None
        self.doc_meta: List[Dict[str, Any]] = []
        self.chunk_meta: List[Dict[str, Any]] = []
        self.chunk_emb: Optional[np.ndarray] = None

    def build(self, uploaded_files: List[st.runtime.uploaded_file_manager.UploadedFile]):
        """Construit l'index à partir des fichiers uploadés"""
        doc_embs, chunk_embs_list = [], []
        
        for doc_id, uf in enumerate(uploaded_files):
            # Sauvegarde temporaire du fichier
            with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp:
                tmp.write(uf.getbuffer())
                path = tmp.name
            
            # SOLUTION: Extraction et traitement du document
            full_text = extract_pdf_text(path)
            os.unlink(path)

            # SOLUTION: Génération du résumé et stockage des métadonnées
            summary = make_summary(full_text)
            self.doc_meta.append({"filename": uf.name, "summary": summary})
            doc_embs.append(embed_texts([summary])[0])

            # SOLUTION: Chunking et embeddings des chunks
            chunks = chunk_document(full_text)
            chunk_embs = embed_texts(chunks)
            chunk_embs_list.append(chunk_embs)
            
            for i, txt in enumerate(chunks):
                self.chunk_meta.append({"doc_id": doc_id, "text": txt, "chunk_id": i})

        # SOLUTION: Création des index FAISS
        self.doc_index = faiss.IndexFlatIP(len(doc_embs[0]))
        self.doc_index.add(np.vstack(doc_embs).astype("float32"))

        self.chunk_emb = np.vstack(chunk_embs_list).astype("float32")
        self.chunk_index = faiss.IndexHNSWFlat(self.chunk_emb.shape[1], 32)
        self.chunk_index.add(self.chunk_emb)

## 9. Détection de questions globales

In [ ]:
def _is_global(self, query: str, thr: float = 0.78) -> bool:
    """Détermine si une question porte sur l'ensemble du document"""
    examples = [
        "De quoi parle ce document ?",
        "Quel est le sujet principal ?",
        "Fais un résumé du document",
    ]
    
    emb_q = embed_texts([query])[0]
    emb_ex = embed_texts(examples)
    
    # Calcul de similarité cosinus
    sims = emb_ex @ emb_q / (np.linalg.norm(emb_ex, axis=1) * np.linalg.norm(emb_q) + 1e-6)
    return float(np.max(sims)) >= thr

# Ajout à la classe RagIndex
RagIndex._is_global = _is_global

## 10. Algorithme MMR - SOLUTION

In [ ]:
# SOLUTION: Implémentation complète de MMR
def _mmr(self, q: np.ndarray, cand: np.ndarray, k: int) -> List[int]:
    """Maximum Marginal Relevance pour diversifier les résultats"""
    selected, rest = [], list(range(len(cand)))
    
    while len(selected) < min(k, len(rest)):
        best, best_score = None, -1e9
        
        for idx in rest:
            # SOLUTION: Calcul de similarité avec la question
            sim_q = float(q @ cand[idx] / (np.linalg.norm(q) * np.linalg.norm(cand[idx]) + 1e-6))
            
            # SOLUTION: Calcul de similarité maximale avec les sélectionnés
            sim_s = max(cosine_similarity(cand[idx][None, :], cand[selected])[0]) if selected else 0.
            
            # SOLUTION: Score MMR
            score = LAMBDA_DIVERSITY * sim_q - (1-LAMBDA_DIVERSITY) * sim_s
            
            if score > best_score:
                best, best_score = idx, score
        
        selected.append(best)
        rest.remove(best)
        
    return selected

# Ajout à la classe RagIndex
RagIndex._mmr = _mmr

## 11. Fonction retrieve - SOLUTION

In [ ]:
# SOLUTION: Implémentation complète de la recherche
def retrieve(self, query: str) -> Tuple[List[str], List[int], Optional[str]]:
    """Recherche les contextes pertinents pour une question"""
    # SOLUTION: Génération de l'embedding de la question
    q_emb = embed_texts([query])[0]
    
    # SOLUTION: Recherche des documents pertinents
    _, I_doc = self.doc_index.search(q_emb[None, :], DOC_TOP_K)
    allowed = set(I_doc[0])

    # SOLUTION: Filtrage des chunks par document
    mask = [i for i, m in enumerate(self.chunk_meta) if m["doc_id"] in allowed]
    sub_emb = self.chunk_emb[mask]
    sub_idx = faiss.IndexFlatIP(sub_emb.shape[1])
    sub_idx.add(sub_emb)
    
    # SOLUTION: Recherche des chunks candidats
    D, I = sub_idx.search(q_emb[None, :], min(CANDIDATES_K, len(mask)))
    pool = [mask[idx] for idx in I[0]]
    
    # SOLUTION: Application du seuil de similarité
    pool = [idx for idx, d in zip(pool, D[0]) if 1-d <= SIM_THRESHOLD] or [mask[I[0][0]]]

    # SOLUTION: Application de MMR
    cand_emb = self.chunk_emb[pool]
    selected = [pool[i] for i in self._mmr(q_emb, cand_emb, CHUNK_TOP_K)]
    
    # SOLUTION: Expansion du contexte
    expanded = {j for idx in selected for j in range(idx-NEIGHBORS, idx+NEIGHBORS+1)}
    final = [i for i in expanded if 0 <= i < len(self.chunk_meta)][:CHUNK_TOP_K]

    # SOLUTION: Construction des contextes finaux
    contexts = [self.chunk_meta[i]["text"] for i in final]
    summary = self.doc_meta[int(I_doc[0][0])]["summary"] if self._is_global(query) else None
    
    return contexts, final, summary

# Ajout à la classe RagIndex
RagIndex.retrieve = retrieve

## 12. Construction des prompts

In [ ]:
def build_prompt(question: str, contexts: List[str], summary: Optional[str], history: List[Dict[str, str]] = None):
    """Construit le prompt pour le modèle de langage"""
    ctx_block = "\n\n".join(f"[{i+1}] {c}" for i, c in enumerate(contexts))
    
    if summary:
        ctx_block = f"[Résumé] {summary}\n\n" + ctx_block
    
    system = (
        "Vous êtes un assistant expert. Utilisez uniquement les informations suivantes pour répondre en français. "
        "Citez vos sources avec les balises [n]. Si l'information n'est pas trouvée, informez-en l'utilisateur."
    )
    
    messages = [{"role": "system", "content": system}]
    
    if history and len(history) > 0:
        previous_messages = history[:-1] if history[-1]["role"] == "user" else history
        messages.extend(previous_messages)
    
    current_user_content = f"CONTEXTE(S):\n{ctx_block}\n\nQUESTION: {question}\n\nRéponse:"
    messages.append({"role": "user", "content": current_user_content})
    
    return messages

## 13. Interface Streamlit

In [ ]:
# Initialisation de l'état Streamlit
if "messages" not in st.session_state:
    st.session_state.messages = []
if "rag" not in st.session_state:
    st.session_state.rag = None
if "processing" not in st.session_state:
    st.session_state.processing = False

# Sidebar pour l'upload de fichiers
with st.sidebar:
    st.header("📚 Documents")
    files = st.file_uploader("Déposez vos PDF", type=["pdf"], accept_multiple_files=True)
    if st.button("🔄 Réinitialiser"):
        st.session_state.clear()
        st.rerun()

# Construction de l'index
if files and st.session_state.rag is None:
    with st.spinner("📄 Indexation en cours…"):
        rag = RagIndex()
        rag.build(files)
        st.session_state.rag = rag
    st.success(f"{len(files)} document(s) indexé(s) ! Posez vos questions.")

## 14. Interface de chat - SOLUTION COMPLÈTE

In [ ]:
# SOLUTION: Interface de chat complète
st.markdown("<div class='chat-container'>", unsafe_allow_html=True)

# Affichage des messages existants
if not st.session_state.messages:
    if st.session_state.rag is not None:
        st.markdown(
            """
            <div class="bot-msg">
            👋 Bonjour ! Je suis votre assistant IA documentaire.
            <br>Posez-moi une question sur le contenu de vos documents.
            </div>
            """,
            unsafe_allow_html=True
        )
    else:
        st.markdown(
            """
            <div class="bot-msg">
            👋 Bienvenue dans RAG PDF Chat !
            <br>Commençez par télécharger un ou plusieurs documents PDF dans le panneau latéral.
            </div>
            """,
            unsafe_allow_html=True
        )
else:
    st.markdown("<div class='chat-area'>", unsafe_allow_html=True)
    for msg in st.session_state.messages:
        css = "user-msg" if msg["role"] == "user" else "bot-msg"
        st.markdown(f'<div class="{css}">{msg["content"]}</div>', unsafe_allow_html=True)
    st.markdown("</div>", unsafe_allow_html=True)

st.markdown("</div>", unsafe_allow_html=True)

# SOLUTION: Gestion de l'input utilisateur
query = st.chat_input("Votre question…", disabled=st.session_state.processing or st.session_state.rag is None)

if query:
    # SOLUTION: Traitement complet de la question
    st.session_state.messages.append({"role": "user", "content": query})
    st.markdown(f'<div class="user-msg">{query}</div>', unsafe_allow_html=True)
    st.session_state.processing = True

    rag: RagIndex = st.session_state.rag
    contexts, indices, summary = rag.retrieve(query)
    
    prompt = build_prompt(query, contexts, summary, st.session_state.messages)

    placeholder = st.empty()
    collected_parts: List[str] = []

    # SOLUTION: Streaming de la réponse
    for chunk in _call_llm(prompt, temperature=TEMPERATURE, max_tokens=MAX_TOKENS, stream=True):
        token = chunk["message"]["content"]
        collected_parts.append(token)
        placeholder.markdown(f'<div class="bot-msg">{"".join(collected_parts)}</div>', unsafe_allow_html=True)

    full_answer = "".join(collected_parts)
    st.session_state.messages.append({"role": "assistant", "content": full_answer})
    st.session_state.processing = False

    # SOLUTION: Affichage des contextes
    with st.expander("🔍 Contextes"):
        for i, ctx in enumerate(contexts):
            st.text_area(f"[{i+1}]", ctx, height=120)

## 15. Tests et validation

In [ ]:
# Tests de validation
print("✅ Configuration terminée")
print(f"📊 Modèle LLM : {MODEL_NAME}")
print(f"🔍 Modèle embeddings : {EMBEDDING_MODEL}")
print("🚀 Application prête pour les tests !")

# Pour Google Colab, utiliser cette commande pour lancer Streamlit :
# !streamlit run formation_rag_chat_corrige.py --server.port 8501 &
# !npx localtunnel --port 8501